In [1]:
import pandas as pd
import os
import glob
import hrvanalysis
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
from hrvanalysis import get_time_domain_features, get_frequency_domain_features
import pandas as pd
import sys

In [2]:
def hrv_analysis(df_ibi):
    rr_intervals_without_outliers = remove_outliers(rr_intervals=df_ibi['ibi'],low_rri=300, high_rri=2000)
    len(rr_intervals_without_outliers)

    # This replace outliers nan values with linear interpolation
    interpolated_rr_intervals = interpolate_nan_values(rr_intervals=rr_intervals_without_outliers, interpolation_method='linear')

    # This remove ectopic beats from signal
    nn_intervals_list = remove_ectopic_beats(rr_intervals=interpolated_rr_intervals)
    nn_intervals_list = interpolate_nan_values(rr_intervals=nn_intervals_list, interpolation_method='linear')

    time_domain_features = get_time_domain_features([x for x in nn_intervals_list if str(x)!='nan'])
    print(time_domain_features)
    frequency_domain_features = get_frequency_domain_features([x for x in nn_intervals_list if str(x)!='nan'])
    print(frequency_domain_features)

# Define the custom format function
def float_format(x):
    return '{:.6f}'.format(x)

In [8]:
#c'è qualcosa di strano con il file 14, tutti gli altri sono stati fatti
for soggetto in range(28, 30):
#for soggetto in range(64, 65):
#for soggetto in range(15, 64):
    # Costruisci il pattern di ricerca per il soggetto corrente
    pattern = f"*subject{soggetto}_*.csv"
    
    # Cerca il file nella cartella tetris corrispondente al soggetto corrente
    percorso_tetris = glob.glob("C:/Users/amary/Flow_experiment/Tetris/" + pattern)
    if percorso_tetris:
        percorso_tetris = percorso_tetris[0]
        
    # Cerca il file nella cartella sensori corrispondente al soggetto corrente
    percorso_sensori = glob.glob("C:/Users/amary/Flow_experiment/Sensori/" + pattern)
    if percorso_sensori:
        percorso_sensori = percorso_sensori[0]

   # print(percorso_tetris)
   # print(percorso_sensori)
    
        # Controlla se entrambi i file esistono
    if percorso_tetris and percorso_sensori:
        # Esegui lo script Python utilizzando i dati dei file
        # Puoi sostituire 'script.py' con il nome del tuo script
        #comando = f"python script.py --tetris {percorso_tetris} --sensori {percorso_sensori}"
        #os.system(comando)
        
        df_ibi = pd.read_csv(percorso_sensori)
        df_ibi = df_ibi.query('ibi != "ibi"')
        
        df_tetris = pd.read_csv(percorso_tetris, header=None)
        #df_tetris = df_tetris[[True if x in ['start_baseline','starting level:49', 'starting level:36', 'starting level:24', 'starting level:12', 'starting level:0'] else False for x in df_tetris[0]]]
        df_tetris = df_tetris[[True if x in ['start_baseline','starting level:49', 'starting level:36', 'starting level:24', 'starting level:14', 'starting level:0'] else False for x in df_tetris[0]]]
        df_tetris = df_tetris.sort_values(1).drop_duplicates(0,keep='first')

        # Set the custom format function for floating-point numbers
        pd.options.display.float_format = float_format
        
        df_ibi['timestamp'] = df_ibi['timestamp'].astype(float)
        df_ibi['ibi'] = df_ibi['ibi'].astype(float)
        
        # Redirecting print output to a file
        sys.stdout = open('C:/Users/amary/Flow_experiment/HRV_analysis/subject' + str(soggetto) + 'new.txt', 'w')
        #sys.stdout = open('C:/Users/amary/Flow_experiment/HRV_analysis/subject'+soggetto+'.txt', 'w')
        start_baseline = float(df_tetris.iloc[0, 1])
        print('start_baseline')
        start_baseline
        print('5_minutes_true_baseline:')
        df_ibi_5min = df_ibi[df_ibi['timestamp'].between(start_baseline,start_baseline+300)]
        hrv_analysis(df_ibi_5min)
        print('')

        for start, name in zip(df_tetris[1][1:], df_tetris[0][1:]):
            start = float(start)
            print(name)
            print(start)
            print()

            print('Baseline:')
            df_ibi_30 = df_ibi[df_ibi['timestamp'].between(start-300, start)]
            hrv_analysis(df_ibi_30)
            print('')

            print('Level:')
            df_ibi_end = df_ibi[df_ibi['timestamp'].between(start, start+180)]
            hrv_analysis(df_ibi_end)
            print('')

            print('================')

        # Close the file
        sys.stdout.close()

        # Reset the standard output
        sys.stdout = sys.__stdout__


In [ ]:
#codici di debug
#vedere se c'è la colonna ibi in tutti i file - ora c'è (c'erano nell'elenco dei file vecchi)

import pandas as pd
import glob
import os
import sys

# Assuming the rest of your imports and setup are correct

for soggetto in range(11, 64):
    # Construct the search pattern for the current subject
    pattern = f"*subject{soggetto}_*.csv"
    
    # Search for the file in the Tetris folder corresponding to the current subject
    percorso_tetris = glob.glob("C:/Users/amary/Flow_experiment/Tetris/" + pattern)
    if percorso_tetris:
        percorso_tetris = percorso_tetris[0]
        
    # Search for the file in the Sensori folder corresponding to the current subject
    percorso_sensori = glob.glob("C:/Users/amary/Flow_experiment/Sensori/" + pattern)
    if percorso_sensori:
        percorso_sensori = percorso_sensori[0]

    # Check if both files exist
    if percorso_tetris and percorso_sensori:
        # Execute the Python script using the data from the files
        comando = f"python script.py --tetris {percorso_tetris} --sensori {percorso_sensori}"
        os.system(comando)
        
        df_ibi = pd.read_csv(percorso_sensori)
        print(df_ibi.columns)  # Print the column names to verify they include 'ibi'
        
        if 'ibi' in df_ibi.columns:
            df_ibi = df_ibi.query('ibi!= "ibi"')
            
            # Continue with your processing...
        else:
            print(f"'ibi' column not found in {percorso_sensori}")

In [ ]:
#debug - vedere se in tutti i file c'è qualcosa - tolti quelli vuoti e doppi

for soggetto in range(11, 64):
    # Costruisci il pattern di ricerca per il soggetto corrente
    pattern = f"*subject{soggetto}_*.csv"
    
    # Cerca il file nella cartella tetris corrispondente al soggetto corrente
    percorso_tetris = glob.glob("C:/Users/amary/Flow_experiment/Tetris/" + pattern)
    if percorso_tetris:
        percorso_tetris = percorso_tetris[0]
        
    # Cerca il file nella cartella sensori corrispondente al soggetto corrente
    percorso_sensori = glob.glob("C:/Users/amary/Flow_experiment/Sensori/" + pattern)
    if percorso_sensori:
        percorso_sensori = percorso_sensori[0]

    print(percorso_tetris)
    print(percorso_sensori)
    
        # Controlla se entrambi i file esistono
  #  if percorso_tetris and percorso_sensori:
        # Esegui lo script Python utilizzando i dati dei file
        # Puoi sostituire 'script.py' con il nome del tuo script
        #comando = f"python script.py --tetris {percorso_tetris} --sensori {percorso_sensori}"
        #os.system(comando)
   
#         print(percorso_tetris)
#         print(percorso_sensori)
    
#         df_ibi = pd.read_csv(percorso_sensori)
#         print(df_ibi.head())
#         df_ibi = df_ibi.query('ibi != "ibi"')
        
#         df_tetris = pd.read_csv(percorso_tetris, header=None)
        
#         try:
#             print(df_test.head())
#         except Exception as e:
#             print(f"Error reading file: {e}")
#        df_tetris = df_tetris[[True if x in ['start_baseline','starting level:49', 'starting level:36', 'starting level:24', 'starting level:12', 'starting level:0'] else False for x in df_tetris[0]]]
#         df_tetris = df_tetris.sort_values(1).drop_duplicates(0,keep='first')

#         # Set the custom format function for floating-point numbers
#         pd.options.display.float_format = float_format
        
#         df_ibi['timestamp'] = df_ibi['timestamp'].astype(float)
#         df_ibi['ibi'] = df_ibi['ibi'].astype(float)
        
#         # Redirecting print output to a file
#         sys.stdout = open('C:/Users/amary/Flow_experiment/HRV_analysis/subject' + str(soggetto) + '.txt', 'w')
#         #sys.stdout = open('C:/Users/amary/Flow_experiment/HRV_analysis/subject'+soggetto+'.txt', 'w')
#         start_baseline = float(df_tetris.iloc[0, 1])
#         print('start_baseline')
#         start_baseline
#         print('5_minutes_true_baseline:')
#         df_ibi_5min = df_ibi[df_ibi['timestamp'].between(start_baseline,start_baseline+300)]
        
        
        
        
        
#         hrv_analysis(df_ibi_5min)
#         print('')

#         for start, name in zip(df_tetris[1][1:], df_tetris[0][1:]):
#             start = float(start)
#             print(name)
#             print(start)
#             print()

#             print('Baseline:')
#             df_ibi_30 = df_ibi[df_ibi['timestamp'].between(start-300, start)]
#             hrv_analysis(df_ibi_30)
#             print('')

#             print('Level:')
#             df_ibi_end = df_ibi[df_ibi['timestamp'].between(start, start+180)]
#             hrv_analysis(df_ibi_end)
#             print('')

#             print('================')

        # Close the file
#        sys.stdout.close()

        # Reset the standard output
#        sys.stdout = sys.__stdout__

In [7]:
#facciamo il soggetto 14

df_ibi = pd.read_csv('dati_IBI_2024-02-03_subject14_task1.csv')


In [8]:
df_ibi 

,ibi,timestamp local,year,month,day,hour,minute,second,millisecond,timestamp
0,714.0,255023473.1276,2024,2,3,18,52,4,358,1706982724.358924
1,ibi,timestamp local,year,month,day,hour,minute,second,millisecond,timestamp
2,716.0,255026571.1097,2024,2,3,18,52,7,457,1706982727.457293
3,ibi,timestamp local,year,month,day,hour,minute,second,millisecond,timestamp
4,716.0,255026573.2262,2024,2,3,18,52,7,459,1706982727.459294
...,...,...,...,...,...,...,...,...,...,...
5200,708.0,257781858.5466,2024,2,3,19,38,2,744,1706985482.744674
5201,ibi,timestamp local,year,month,day,hour,minute,second,millisecond,timestamp
5202,714.0,257781868.2663,2024,2,3,19,38,2,753,1706985482.75368
5203,ibi,timestamp local,year,month,day,hour,minute,second,millisecond,timestamp


In [9]:
df_ibi = df_ibi.query('ibi != "ibi"')
df_ibi

,ibi,timestamp local,year,month,day,hour,minute,second,millisecond,timestamp
0,714.0,255023473.1276,2024,2,3,18,52,4,358,1706982724.358924
2,716.0,255026571.1097,2024,2,3,18,52,7,457,1706982727.457293
4,716.0,255026573.2262,2024,2,3,18,52,7,459,1706982727.459294
6,710.0,255030928.5781,2024,2,3,18,52,11,814,1706982731.814011
8,709.0,255035215.9324,2024,2,3,18,52,16,101,1706982736.102303
...,...,...,...,...,...,...,...,...,...,...
5196,693.0,257780857.942,2024,2,3,19,38,1,743,1706985481.743835
5198,703.0,257780865.8917,2024,2,3,19,38,1,751,1706985481.751831
5200,708.0,257781858.5466,2024,2,3,19,38,2,744,1706985482.744674
5202,714.0,257781868.2663,2024,2,3,19,38,2,753,1706985482.75368


In [10]:
df_ibi['ibi']

0       714.0
2       716.0
4       716.0
6       710.0
8       709.0
        ...  
5196    693.0
5198    703.0
5200    708.0
5202    714.0
5204    721.0
Name: ibi, Length: 2603, dtype: object

In [11]:
df_ibi['timestamp']

0       1706982724.358924
2       1706982727.457293
4       1706982727.459294
6       1706982731.814011
8       1706982736.102303
              ...        
5196    1706985481.743835
5198    1706985481.751831
5200    1706985482.744674
5202     1706985482.75368
5204    1706985483.744297
Name: timestamp, Length: 2603, dtype: object

In [12]:
#perché questi?
#df_ibi = df_ibi[df_ibi['timestamp'].astype(float).diff()<2]
#df_ibi['ibi'] = df_ibi['ibi'].astype(float)
#df_ibi['timestamp'] = df_ibi['timestamp'].astype(float)

In [13]:
df_ibi[df_ibi['timestamp'].astype(float).diff()>2]

,ibi,timestamp local,year,month,day,hour,minute,second,millisecond,timestamp
2,716.0,255026571.1097,2024,2,3,18,52,7,457,1706982727.457293
6,710.0,255030928.5781,2024,2,3,18,52,11,814,1706982731.814011
8,709.0,255035215.9324,2024,2,3,18,52,16,101,1706982736.102303
12,706.0,255038771.8206,2024,2,3,18,52,19,657,1706982739.657985
14,704.0,255042926.9164,2024,2,3,18,52,23,812,1706982743.812468
22,742.0,255875957.458,2024,2,3,19,6,16,843,1706983576.843627
920,747.0,256212356.2637,2024,2,3,19,11,53,242,1706983913.242515
1784,725.0,256528359.1794,2024,2,3,19,17,9,245,1706984229.245196
2680,718.0,256853856.7796,2024,2,3,19,22,34,742,1706984554.742907
3098,747.0,257007360.0898,2024,2,3,19,25,8,245,1706984708.24564


In [14]:
df_tetris = pd.read_csv('C:/Users/amary/Flow_experiment/Tetris/dati_tetris_2024-02-03_subject14_task1.csv',header=None)
df_tetris = df_tetris[[True if x in ['start_baseline','starting level:49', 'starting level:36', 'starting level:24', 'starting level:12', 'starting level:0'] else False for x in df_tetris[0]]]
df_tetris = df_tetris.sort_values(1).drop_duplicates(0,keep='first')

In [15]:
df_tetris

,0,1,2,3
0,start_baseline,1706982868.151873,255145984,255167.266495
4,starting level:36,1706983212.479347,255490312,255511.593927
31,starting level:12,1706983716.452484,255994281,256015.567123
42,starting level:49,1706984211.488634,256489328,256510.602652
93,starting level:24,1706984707.984804,256985828,257007.099284
116,starting level:0,1706985203.386904,257481218,257502.501124


In [16]:
df_tetris[1]

0     1706982868.151873
4     1706983212.479347
31    1706983716.452484
42    1706984211.488634
93    1706984707.984804
116   1706985203.386904
Name: 1, dtype: float64

In [17]:
import pandas as pd

# Define the custom format function
def float_format(x):
    return '{:.6f}'.format(x)

# Set the custom format function for floating-point numbers
pd.options.display.float_format = float_format

# Now, when you display the DataFrame column, the values will be in numeric format
print(df_tetris)

In [18]:
df_tetris

,0,1,2,3
0,start_baseline,1706982868.151873,255145984,255167.266495
4,starting level:36,1706983212.479347,255490312,255511.593927
31,starting level:12,1706983716.452484,255994281,256015.567123
42,starting level:49,1706984211.488634,256489328,256510.602652
93,starting level:24,1706984707.984804,256985828,257007.099284
116,starting level:0,1706985203.386904,257481218,257502.501124


In [ ]:
# 
Devi prendee anche le baseline!

In [19]:
df_ibi['timestamp'] = df_ibi['timestamp'].astype(float)
df_ibi['timestamp'] 

C:\Users\amary\AppData\Local\Temp\ipykernel_14956\3135384406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ibi['timestamp'] = df_ibi['timestamp'].astype(float)


0      1706982724.358924
2      1706982727.457293
4      1706982727.459294
6      1706982731.814011
8      1706982736.102303
              ...       
5196   1706985481.743835
5198   1706985481.751831
5200   1706985482.744674
5202   1706985482.753680
5204   1706985483.744297
Name: timestamp, Length: 2603, dtype: float64

In [ ]:
#non c'è la baseline - non è stata registrata
start_baseline = float(df_tetris.iloc[0, 1])
start_baseline
df_ibi_5min = df_ibi[df_ibi['timestamp'].between(start_baseline,start_baseline+300)]
df_ibi_5min

In [ ]:
start_baseline

In [ ]:
start_baseline+300

In [ ]:
df_ibi2['timestamp']

In [20]:
start_baseline = float(df_tetris.iloc[0, 1])
start_baseline
print('5_minutes_true_baseline:')
df_ibi_5min = df_ibi[df_ibi['timestamp'].between(start_baseline,start_baseline+300)]
hrv_analysis(df_ibi_5min)
print('')

C:\Users\amary\AppData\Roaming\Python\Python311\site-packages\hrvanalysis\preprocessing.py:286: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series_rr_intervals_cleaned = pd.Series(rr_intervals)


IndexError: list index out of range

In [ ]:
df_ibi['timestamp'] = df_ibi['timestamp'].astype(float)
df_ibi['ibi'] = df_ibi['ibi'].astype(float)

start_baseline = float(df_tetris.iloc[0, 1])
    print(name)
    print(start)
    print()
    
    print('Baseline:')
    df_ibi_30 = df_ibi[df_ibi['timestamp'].between(start-300,start)]
    hrv_analysis(df_ibi_30)
    print('')
    
    print('Level:')
    df_ibi_end = df_ibi[df_ibi['timestamp'].between(start,start+180)]
    hrv_analysis(df_ibi_end)
    print('')
    
    print('================')


for start,name in zip(df_tetris[1],df_tetris[0]):
    start = float(start)
    print(name)
    print(start)
    print()
    
    print('Baseline:')
    df_ibi_30 = df_ibi[df_ibi['timestamp'].between(start-300,start)]
    hrv_analysis(df_ibi_30)
    print('')
    
    print('Level:')
    df_ibi_end = df_ibi[df_ibi['timestamp'].between(start,start+180)]
    hrv_analysis(df_ibi_end)
    print('')
    
    print('================')

In [25]:
import sys

# Redirecting print output to a file
sys.stdout = open('C:/Users/amary/Flow_experiment/HRV_analysis/subject14_sololivelli.txt', 'w')

# Your code
df_ibi['timestamp'] = df_ibi['timestamp'].astype(float)
df_ibi['ibi'] = df_ibi['ibi'].astype(float)

#start_baseline = float(df_tetris.iloc[0, 1])
print('start_baseline')
#start_baseline
print('5_minutes_true_baseline:')
#df_ibi_5min = df_ibi[df_ibi['timestamp'].between(start_baseline,start_baseline+300)]
#hrv_analysis(df_ibi_5min)
print('missing')
print('')

for start, name in zip(df_tetris[1][2:], df_tetris[0][2:]):
    start = float(start)
    print(name)
    print(start)
    print()

    print('Baseline:')
    df_ibi_30 = df_ibi[df_ibi['timestamp'].between(start-300, start)]
    hrv_analysis(df_ibi_30)
    print('')

    print('Level:')
    df_ibi_end = df_ibi[df_ibi['timestamp'].between(start, start+180)]
    hrv_analysis(df_ibi_end)
    print('')

    print('================')

# Close the file
sys.stdout.close()

# Reset the standard output
sys.stdout = sys.__stdout__

C:\Users\amary\AppData\Local\Temp\ipykernel_14956\1354220101.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ibi['timestamp'] = df_ibi['timestamp'].astype(float)
C:\Users\amary\AppData\Local\Temp\ipykernel_14956\1354220101.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ibi['ibi'] = df_ibi['ibi'].astype(float)


In [23]:
df_tetris[1][2:]

31    1706983716.452484
42    1706984211.488634
93    1706984707.984804
116   1706985203.386904
Name: 1, dtype: float64

In [24]:
df_tetris[0][2:]

31     starting level:12
42     starting level:49
93     starting level:24
116     starting level:0
Name: 0, dtype: object